# 3. 並行鏈 (Chains Parallel) - Gemini 版本

本範例展示如何同時執行多個分支，建立一個簡單的餐廳評論分析系統，同時分析優點和缺點。

## 學習重點
- 使用 RunnableParallel 實現並行處理
- 學習如何同時執行多個分析任務
- 理解並行處理的優勢
- 掌握複雜鏈的組合技巧


In [ ]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_google_genai import ChatGoogleGenerativeAI

# 載入環境變數
load_dotenv()

# 建立 Gemini 模型
model = ChatGoogleGenerativeAI(model="gemini-flash-2.5")


In [ ]:
# 定義主要的提示模板 - 餐廳評論分析
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的餐廳評論家，擅長分析餐廳的優缺點。"),
        ("human", "請分析這家餐廳：{restaurant_name}"),
    ]
)


In [ ]:
# 定義優點分析函數
def analyze_pros(restaurant_info):
    """分析餐廳的優點"""
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的優點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的優點。"),
        ]
    )
    return pros_template.format_prompt(restaurant_info=restaurant_info)

# 定義缺點分析函數
def analyze_cons(restaurant_info):
    """分析餐廳的缺點"""
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "你是一個專業的餐廳評論家，專門分析餐廳的缺點。"),
            ("human", "根據餐廳資訊：{restaurant_info}，請列出這家餐廳的缺點。"),
        ]
    )
    return cons_template.format_prompt(restaurant_info=restaurant_info)

print("分析函數已定義：")
print("1. analyze_pros: 分析餐廳優點")
print("2. analyze_cons: 分析餐廳缺點")


In [ ]:
# 定義結果合併函數
def combine_pros_cons(pros, cons):
    """將優點和缺點合併成最終評論"""
    return f"""
🍽️ 餐廳評論分析報告
{'='*50}

✅ 優點分析：
{pros}

❌ 缺點分析：
{cons}

📊 總結：這是一個平衡的餐廳評論分析
"""

print("結果合併函數已定義：combine_pros_cons")


In [ ]:
# 使用 LCEL 簡化分支鏈的建立
# 優點分析分支
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

# 缺點分析分支
cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

print("並行分支鏈已建立：")
print("1. pros_branch_chain: 餐廳優點分析分支")
print("2. cons_branch_chain: 餐廳缺點分析分支")


In [ ]:
# 建立完整的並行處理鏈
# 修正：RunnableParallel 直接使用鍵值對，不需要 branches 包裝
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(
        pros=pros_branch_chain,
        cons=cons_branch_chain
    )
    | RunnableLambda(lambda x: combine_pros_cons(x["pros"], x["cons"]))
)

print("並行處理鏈已建立完成！")
print("鏈的流程：餐廳資訊 → 並行分析(優點+缺點) → 合併結果")

In [ ]:
# 執行並行處理鏈 - 測試餐廳評論分析系統
restaurant_info = """
這是一家位於台北市中心的義大利餐廳，提供傳統義式料理。
餐廳環境優雅，服務人員親切，但價格偏高，等待時間較長。
食物品質不錯，但份量偏少。適合約會或特殊場合用餐。
"""

result = chain.invoke({"restaurant_name": "義大利風情餐廳"})

# 輸出結果
print("=" * 60)
print("餐廳評論分析結果：")
print("=" * 60)
print(result)


## 💡 重點說明

### 並行處理的優勢

1. **效率提升**: 同時執行多個分析任務，節省時間
2. **資源利用**: 充分利用系統資源，提高處理速度
3. **模組化**: 每個分支都是獨立的，易於維護和修改

### RunnableParallel 的使用

```python
# 正確語法：直接使用鍵值對參數
RunnableParallel(
    pros=pros_branch_chain,
    cons=cons_branch_chain
)

# 輸出格式：{"pros": "...", "cons": "..."}
```

**常見錯誤**：
```python
# ❌ 錯誤：不要使用 branches={} 包裝
RunnableParallel(branches={"pros": ..., "cons": ...})

# ✅ 正確：直接傳入鍵值對
RunnableParallel(pros=..., cons=...)
```

### 並行鏈的資料流

1. **輸入**: 餐廳資訊
2. **初步分析**: 生成餐廳基本資訊
3. **並行處理**: 同時分析優點和缺點
4. **結果合併**: 將並行結果整合成最終報告

### 實際應用場景

- **餐廳評論**: 同時分析優缺點
- **產品評測**: 多角度分析產品
- **內容分析**: 並行處理不同面向
- **數據處理**: 同時執行多個任務